In [1]:
!pip install gradio


In [2]:
import gradio as gr
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
from tensorflow.keras.layers import GRU, Dense, Embedding

In [8]:
text = """
Once upon a time in a land far away, there was a little village surrounded by lush green forests.
The villagers lived in harmony with nature, cherishing the beauty around them.
"""

In [9]:
tokenizer = Tokenizer(char_level=True)  # Tokenize at the character level
tokenizer.fit_on_texts(text)
char_index = tokenizer.word_index
index_char = tokenizer.index_word
vocab_size = len(char_index) + 1

In [10]:
sequences = tokenizer.texts_to_sequences(text)

# Flatten the list of sequences
sequences = [item for sublist in sequences for item in sublist]

# Create input-output pairs
seq_length = 40
X, y = [], []
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

In [11]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    GRU(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X, y, epochs=100, batch_size=64)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0446 - loss: 3.2551
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1428 - loss: 3.2359
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1916 - loss: 3.2180
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2128 - loss: 3.1886
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.1972 - loss: 3.1559
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1564 - loss: 3.0807
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1467 - loss: 2.9683
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.1369 - loss: 2.9409
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step - accuracy: 0.1408 - loss: 2.9281
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.1916 - loss: 2.9306
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.2203 - loss: 2.9206
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2256 - 

In [13]:
def generate_text(seed_text, num_chars=200):
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, truncating='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_char_index = np.argmax(predicted_probs)
        predicted_char = index_char[predicted_char_index]
        seed_text += predicted_char
    return seed_text

In [14]:
def gradio_generate(seed_text, num_chars):
    return generate_text(seed_text, int(num_chars))

interface = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Seed Text", placeholder="Enter seed text, e.g., 'Once upon a time'"),
        gr.Slider(minimum=50, maximum=500, step=10, value=200, label="Number of Characters")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GRU-Based Text Generator",
    description="Enter a seed text and specify the number of characters to generate new text using a GRU-based neural network."
)

# Launch the Gradio app
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c7fa75c172674c662.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
